In [27]:
import pymysql
import csv
import os
from dotenv import load_dotenv
load_dotenv()
MYSQL_ROOT_PW = os.environ.get("MYSQL_ROOT_PW")

In [28]:
con = pymysql.connect(host="localhost", db="ecommerce", user="root", password=MYSQL_ROOT_PW)
cur = con.cursor()

In [29]:
# data 디렉토리의 csv파일들을 mysql 서버에 적재하기 위한 CREATE TABLE DDL query 작성
# 데이터 타입을 고려하여 schema를 설계한다

customer_ddl = """
CREATE TABLE customer
(
    customer_id VARCHAR(10) NOT NULL,
    gender VARCHAR(2),
    city VARCHAR(20),
    duration INT,
    PRIMARY KEY (customer_id)
);
"""

discount_ddl = """
CREATE TABLE discount
(
    month VARCHAR(5) NOT NULL,
    category VARCHAR(20) NOT NULL,
    coupon_code VARCHAR(10) NOT NULL,
    discount_rate FLOAT
);
"""

marketing_ddl = """
CREATE TABLE marketing
(
    marketing_date DATE NOT NULL,
    offline_expense FLOAT,
    online_expense FLOAT,
    PRIMARY KEY (marketing_date)
);
"""

onlinesales_ddl = """
CREATE TABLE onlinesales
(
    customer_id VARCHAR(10) NOT NULL,
    transaction_id VARCHAR(30) NOT NULL,
    transaction_date DATE,
    product_id VARCHAR(20) NOT NULL,
    category VARCHAR(20),
    amount INT,
    average_price FLOAT,
    shipping_fee FLOAT,
    coupon_status VARCHAR(20)
);
"""

tax_ddl = """
CREATE TABLE tax
(
    category VARCHAR(20),
    GST FLOAT
);
"""


ddl_list = [customer_ddl, discount_ddl, marketing_ddl, onlinesales_ddl, tax_ddl]
for ddl in ddl_list: cur.execute(ddl)

In [30]:
def insert_data(path, table, n_columns):
    f = open(os.path.join("data", path), "r")
    reader = csv.reader(f)
    next(reader, None)
    for row in reader:
        query = f"INSERT INTO {table} VALUES({', '.join(['%s' for x in range(n_columns)])})"
        cur.execute(query, tuple(row))
    con.commit()
    f.close()

In [31]:
data_path = os.listdir("data")
insert_data(data_path[0], "customer", 4)
insert_data(data_path[1], "discount", 4)
insert_data(data_path[2], "marketing", 3)
insert_data(data_path[3], "onlinesales", 9)
insert_data(data_path[4], "tax", 2)